In [177]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [178]:
#flask_api\model.py

import torch
import torch.nn as nn
import os
import pickle
import numpy as np
from torchvision import transforms, models
from transformers import DeiTForImageClassification, CLIPModel, CLIPProcessor
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

# Load SimCLR Model
class SimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(SimCLR, self).__init__()
        self.backbone = nn.Sequential(*list(base_model.children())[:-1])
        self.projection_head = nn.Sequential(
            nn.Linear(base_model.fc.in_features, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def forward(self, x):
        h = self.backbone(x).squeeze()
        z = self.projection_head(h)
        return h, z

def load_model(model_path, model):
    # Corrected: No 'weights_only' argument in load_state_dict
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
    model.eval()  # Set to evaluation mode
    return model

# Load models
def load_models():
    simclr_model_instance = SimCLR(models.resnet50(weights='ResNet50_Weights.DEFAULT'), out_dim=128)
    simclr_model = load_model('models/simclr_model.pth', simclr_model_instance)

    deit_model = DeiTForImageClassification.from_pretrained('facebook/deit-base-distilled-patch16-224', num_labels=128)
    deit_model.load_state_dict(torch.load('models/deit_model.pth', map_location=torch.device('cpu')))

    clip_model = load_model('models/clip_model.pth', CLIPModel.from_pretrained("openai/clip-vit-base-patch32"))
    
    base_cnn_model = models.resnet18(weights='ResNet18_Weights.DEFAULT')  # Load default ResNet18
    base_cnn_model.eval()  # Set to evaluation mode

    return simclr_model, deit_model, clip_model, base_cnn_model

# Image transform for non-CLIP models
def get_image_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

# # Feature extraction functions
# def extract_features(model, image, model_type):
#     with torch.no_grad():
#         image_tensor = get_image_transform()(image).unsqueeze(0)
#         if model_type == 'simclr':
#             return model(image_tensor)[1]
#         elif model_type == 'deit':
#             inputs = {"pixel_values": image_tensor}
#             outputs = model(**inputs)
#             return outputs.logits
#         elif model_type == 'clip':
#             processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
#             clip_inputs = processor(images=image, return_tensors="pt")
#             image_features = model.get_image_features(**clip_inputs)
#             return image_features.cpu().numpy().reshape(1, -1)
#         elif model_type == 'base_cnn':
#             return model(image_tensor).detach().cpu().numpy()

def extract_features(model, image_tensor, model_type):
    """Extract features from the given model."""
    with torch.no_grad():
        if model_type == "cnn":
            return model(image_tensor).squeeze().cpu().numpy()
        elif model_type == "clip":
            return model.get_image_features(image_tensor).squeeze().cpu().numpy()
        elif model_type == "deit":
            return model(image_tensor).logits.squeeze().cpu().numpy()


# Load and save features
def load_features(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            features = pickle.load(f)
        print(f"Loaded features from {filename}.")
        return features
    return {}

def save_features_to_file(features, filename):
    with open(filename, 'wb') as f:
        pickle.dump(features, f)
    print(f"Features saved to {filename}.")

# Find most similar images: Eliana (original)
# def find_most_similar_image(uploaded_features, dataset_features):
#     uploaded_features = uploaded_features.reshape(1, -1)  # Ensure shape is [1, feature_dim]
#     dataset_features_array = np.array(list(dataset_features.values()))
#     dataset_features_array = dataset_features_array.reshape(dataset_features_array.shape[0], -1)

#     similarities_cosine = cosine_similarity(uploaded_features, dataset_features_array)

#     most_similar = sorted(zip(dataset_features.keys(), similarities_cosine[0]),
#                           key=lambda item: item[1], reverse=True)

#     return most_similar

#  Revised Working Attempt (by Alleina)
# def find_most_similar_image(uploaded_features, dataset_features):
#     """Find most similar images from dataset features."""
#     uploaded_features = uploaded_features.reshape(1, -1)  # Ensure query feature is [1, dim]
#     dataset_features_array = np.array(list(dataset_features.values()))
#     dataset_features_array = dataset_features_array.reshape(dataset_features_array.shape[0], -1)

#     if uploaded_features.shape[1] != dataset_features_array.shape[1]:
#         raise ValueError(
#             f"Dimension mismatch: Query features have {uploaded_features.shape[1]} dimensions, "
#             f"but dataset features have {dataset_features_array.shape[1]} dimensions."
#         )

#     similarities_cosine = cosine_similarity(uploaded_features, dataset_features_array)
#     most_similar = sorted(
#         zip(dataset_features.keys(), similarities_cosine[0]),
#         key=lambda item: item[1],
#         reverse=True,
#     )

#     return most_similar

# Removed .values() functions
# def find_most_similar_image(uploaded_features, dataset_features):
#     """Find most similar images from dataset features."""
#     uploaded_features = uploaded_features.reshape(1, -1)  # Ensure query feature is [1, dim]
#     dataset_features_array = np.array(dataset_features)  # No need for .values() on numpy arrays
#     dataset_features_array = dataset_features_array.reshape(dataset_features_array.shape[0], -1)
    
#     if uploaded_features.shape[1] != dataset_features_array.shape[1]:
#         raise ValueError(f"Feature dimension mismatch: query features {uploaded_features.shape[1]}, dataset features {dataset_features_array.shape[1]}")

#     # Calculate similarity between query and dataset features (e.g., cosine similarity)
#     similarity_scores = cosine_similarity(uploaded_features, dataset_features_array)
    
#     # Get the indices of the most similar images
#     most_similar_indices = np.argsort(similarity_scores[0])[::-1]
#     similar_images = [(dataset_features[i], similarity_scores[0][i]) for i in most_similar_indices]
    
#     return similar_images

# from sklearn.preprocessing import StandardScaler

# def find_most_similar_image(uploaded_features, dataset_features):
#     """Find most similar images from dataset features."""
#     uploaded_features = uploaded_features.reshape(1, -1)  # Ensure query feature is [1, dim]
    
#     # Check if feature dimension mismatch exists
#     if uploaded_features.shape[1] != dataset_features.shape[1]:
#         if uploaded_features.shape[1] == 512 and dataset_features.shape[1] == 768:
#             # Rescale query feature to match dataset feature dimension (768)
#             print("Rescaling query feature from 512 to 768...")
#             uploaded_features = np.resize(uploaded_features, (uploaded_features.shape[0], 768))  # Resizing query features to 768
            
#         else:
#             raise ValueError(f"Feature dimension mismatch: query features {uploaded_features.shape[1]}, dataset features {dataset_features.shape[1]}")
    
#     # Calculate similarity between query and dataset features (e.g., cosine similarity)
#     similarity_scores = cosine_similarity(uploaded_features, dataset_features)
    
#     # Get the indices of the most similar images
#     most_similar_indices = np.argsort(similarity_scores[0])[::-1]
#     similar_images = [(dataset_features[i], similarity_scores[0][i]) for i in most_similar_indices]
    
#     return similar_images

def find_most_similar_image(uploaded_features, dataset_features, image_paths):
    """Find most similar images from dataset features."""
    uploaded_features = uploaded_features.reshape(1, -1)  # Ensure query feature is [1, dim]
    
    # Check if feature dimension mismatch exists
    if uploaded_features.shape[1] != dataset_features.shape[1]:
        if uploaded_features.shape[1] == 512 and dataset_features.shape[1] == 768:
            # Rescale query feature to match dataset feature dimension (768)
            print("Rescaling query feature from 512 to 768...")
            uploaded_features = np.resize(uploaded_features, (uploaded_features.shape[0], 768))  # Resizing query features to 768
            
        else:
            raise ValueError(f"Feature dimension mismatch: query features {uploaded_features.shape[1]}, dataset features {dataset_features.shape[1]}")
    
    # Calculate similarity between query and dataset features (e.g., cosine similarity)
    similarity_scores = cosine_similarity(uploaded_features, dataset_features)
    
    # Get the indices of the most similar images
    most_similar_indices = np.argsort(similarity_scores[0])[::-1]
    similar_images = [(image_paths[i], similarity_scores[0][i]) for i in most_similar_indices]
    
    return similar_images




# Load dataset features
def load_dataset_features(image_folder, model, model_type, feature_file):
    features = load_features(feature_file)
    if features:
        return features  # Skip extraction if features are already loaded

    features = {}
    img_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.png'))]

    print(f"Loading features for {len(img_files)} images...")
    
    for img_file in img_files:
        img_path = os.path.join(image_folder, img_file)
        image = Image.open(img_path).convert("RGB")
        feature = extract_features(model, image, model_type)
        features[img_path] = feature

    save_features_to_file(features, feature_file)
    return features

In [179]:
import os
import csv
import pickle
from PIL import Image
# from ImageMatcher import load_models, find_most_similar_image, get_image_transform, extract_features

In [180]:
# Paths
QUERY_IMAGE_PATH = r"C:\Users\lenovo\OneDrive\Documents\PROJECTS\ALT\Real-Estate-Image-Matching\evaluation\image-matching\query\listing_1_image_1.png_20240927_122438.png"  # Path to the query image
OUTPUT_FOLDER = r"output"  # Output folder for the CSV files

In [181]:
# Feature files
CNN_FEATURES_FILE = r'C:\Users\lenovo\OneDrive\Documents\PROJECTS\ALT\Real-Estate-Image-Matching\image-matching\features\cnn features.pkl'
CLIP_FEATURES_FILE = r'C:\Users\lenovo\OneDrive\Documents\PROJECTS\ALT\Real-Estate-Image-Matching\image-matching\features\clip features.pkl'
DEIT_FEATURES_FILE = r'C:\Users\lenovo\OneDrive\Documents\PROJECTS\ALT\Real-Estate-Image-Matching\image-matching\features\deit features.pkl'

In [182]:
import pickle

with open(DEIT_FEATURES_FILE, "rb") as f:
    dataset_features_clip = pickle.load(f)

for k, v in dataset_features_clip.items():
    print(f"Path: {k}, Feature Shape: {v.shape}")


Path: C:\images\listing_1000_image_1.png_20240927_173827.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_1.png_20240927_125602.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_1.png_20240927_183330.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_2.png_20240927_125604.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_2.png_20240927_183331.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_3.png_20240927_125605.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_3.png_20240927_183333.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_4.png_20240927_183335.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_5.png_20240927_183336.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_image_6.png_20240927_183338.png, Feature Shape: torch.Size([1, 128])
Path: C:\images\listing_100_i

In [183]:
from sklearn.decomposition import PCA
import numpy as np

def reshape_features(features, expected_dim):
    features = np.array(features)
    print(f"Features shape before reshape: {features.shape}")

    # Handle 3D array by removing the second dimension (e.g., (8158, 1, 1000) -> (8158, 1000))
    if len(features.shape) == 3:
        features = features.squeeze(axis=1)

    # If the features are 1D (like in the case of query image features), reshape to 2D
    if len(features.shape) == 1:
        features = features.reshape(1, -1)
    
    # If features are 2D but have an unexpected second dimension (e.g., (1, 1000) -> reshape it to (1, 1000))
    elif len(features.shape) == 2 and features.shape[1] != expected_dim:
        features = features.reshape(features.shape[0], -1)

    # If the features' dimension is larger than expected (e.g., from 768 to 512 for CLIP), apply PCA to reduce the dimensionality
    if features.shape[1] > expected_dim:
        print(f"Reducing features from {features.shape[1]} to {expected_dim} using PCA")
        pca = PCA(n_components=expected_dim)
        features = pca.fit_transform(features)
    
    # Check if the final shape matches the expected dimension
    if features.shape[1] != expected_dim:
        raise ValueError(f"Expected features with dimension {expected_dim}, but got {features.shape[1]}")

    print(f"Features shape after reshape: {features.shape}")
    return features


In [184]:
# Ensure output folder exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [185]:
def preprocess_image(image, model_type):
    """Preprocess the image based on the model type."""
    transform = get_image_transform()
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    if model_type == "clip":
        from transformers import CLIPProcessor
        processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        clip_inputs = processor(images=image, return_tensors="pt")
        return clip_inputs["pixel_values"]
    return image_tensor


In [186]:
def load_features_from_file(feature_file):
    """Load precomputed features from a .pkl file."""
    with open(feature_file, 'rb') as f:
        features = pickle.load(f)
    return features

In [187]:
# def save_to_csv(filename, query_image, similar_images, similarity_scores):
#     """Save results to a CSV file."""
#     filepath = os.path.join(OUTPUT_FOLDER, filename)
#     with open(filepath, mode='w', newline='', encoding='utf-8') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerow(["Query Image", "Similar Images", "Similarity Scores"])
#         writer.writerow([
#             query_image,
#             ", ".join(similar_images),
#             ", ".join(similarity_scores)
#         ])
#     print(f"Results saved to {filepath}")

def save_to_csv(filename, query_image, similar_images, similarity_scores):
    """Save results to a CSV file."""
    filepath = os.path.join(OUTPUT_FOLDER, filename)
    with open(filepath, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Query Image", "Similar Images", "Similarity Scores"])
        writer.writerow([
            query_image,
            ", ".join(similar_images),  # Join image paths as a single string
            ", ".join(similarity_scores)  # Join similarity scores as a single string
        ])
    print(f"Results saved to {filepath}")


In [188]:
from sklearn.decomposition import PCA

In [189]:
def main():
    # Load models (used only for extracting query image features)
    _, deit_model, clip_model, cnn_model = load_models()

    # Load query image
    query_image = Image.open(QUERY_IMAGE_PATH).convert("RGB")
    print("Query Image loaded")

    # Extract query image features
    print("\nExtracting query image features using CNN model...")
    query_features_cnn = extract_features(cnn_model, preprocess_image(query_image, "cnn"), "cnn")
    query_features_cnn = reshape_features(query_features_cnn, 1000)
    
    print("\nExtracting query image features using CLIP model...")
    query_features_clip = extract_features(clip_model, preprocess_image(query_image, "clip"), "clip")
    query_features_clip = reshape_features(query_features_clip, 512)

    print("\nExtracting query image features using DEIT model...")
    query_features_deit = extract_features(deit_model, preprocess_image(query_image, "deit"), "deit")
    query_features_deit = reshape_features(query_features_deit, 128)

    # Load precomputed dataset features
    print("\nLeading pre-computed CNN Features File...")
    dataset_features_cnn = load_features_from_file(CNN_FEATURES_FILE)
    dataset_features_cnn = reshape_features(np.array(list(dataset_features_cnn.values())), 1000)
    print("\nLeading pre-computed CLIP Features File...")
    dataset_features_clip = load_features_from_file(CLIP_FEATURES_FILE)
    dataset_features_clip = reshape_features(np.array(list(dataset_features_clip.values())), 512)

    print("\nLeading pre-computed DEIT Features File...")
    dataset_features_deit = load_features_from_file(DEIT_FEATURES_FILE)
    dataset_features_deit = reshape_features(np.array(list(dataset_features_deit.values())), 128)

    # Load image paths for dataset
    image_paths_cnn = list(dataset_features_cnn.keys())
    image_paths_clip = list(dataset_features_clip.keys())
    image_paths_deit = list(dataset_features_deit.keys())

    # Find similar images for each model
    print("Finding most similar images using CNN...")
    similar_images_cnn = find_most_similar_image(query_features_cnn, dataset_features_cnn)
    print("Finding most similar images using CLIP...")
    similar_images_clip = find_most_similar_image(query_features_clip, dataset_features_clip)
    print("Finding most similar images using DEIT...")
    similar_images_deit = find_most_similar_image(query_features_deit, dataset_features_deit)

    # Combine CLIP and DEIT results
    combined_clip_deit = similar_images_clip + similar_images_deit
    print("Combining CLIP and DEIT results...")
    combined_clip_deit = sorted(combined_clip_deit, key=lambda x: x[1], reverse=True)[:5]
    print(combined_clip_deit)

    # Prepare data for CSVs
    # def prepare_csv_data(results):
    #     similar_images = [str(img) for img, _ in results]  # Convert each image path (or object) to string
    #     similarity_scores = [f"{score:.4f}" for _, score in results]
    #     return similar_images, similarity_scores

    def prepare_csv_data(results):
        """Prepare similar images and similarity scores for CSV."""
        similar_images = [img for img, _ in results]  # Only image paths
        similarity_scores = [f"{score:.4f}" for _, score in results]  # Format similarity scores
        return similar_images, similarity_scores


    # similar_images_cnn_paths, similarity_scores_cnn = prepare_csv_data(similar_images_cnn[:5])
    # print(similar_images_cnn_paths, similarity_scores_cnn)

    # similar_images_clip_paths, similarity_scores_clip = prepare_csv_data(similar_images_clip[:5])
    # print(similar_images_clip_paths, similarity_scores_clip)

    # similar_images_deit_paths, similarity_scores_deit = prepare_csv_data(similar_images_deit[:5])
    # print(similar_images_deit_paths, similarity_scores_deit)
    
    # combined_clip_deit_paths, combined_clip_deit_scores = prepare_csv_data(combined_clip_deit)

    # Find similar images for each model
    print("Finding most similar images using CNN...")
    similar_images_cnn = find_most_similar_image(query_features_cnn, dataset_features_cnn, image_paths_cnn)
    print("Finding most similar images using CLIP...")
    similar_images_clip = find_most_similar_image(query_features_clip, dataset_features_clip, image_paths_clip)
    print("Finding most similar images using DEIT...")
    similar_images_deit = find_most_similar_image(query_features_deit, dataset_features_deit, image_paths_deit)

    # Combine CLIP and DEIT results
    combined_clip_deit = similar_images_clip + similar_images_deit
    print("Combining CLIP and DEIT results...")
    combined_clip_deit = sorted(combined_clip_deit, key=lambda x: x[1], reverse=True)[:5]
    print(combined_clip_deit)

    # Prepare data for CSVs
    similar_images_cnn_paths, similarity_scores_cnn = prepare_csv_data(similar_images_cnn[:5])
    print(similar_images_cnn_paths, similarity_scores_cnn)

    similar_images_clip_paths, similarity_scores_clip = prepare_csv_data(similar_images_clip[:5])
    print(similar_images_clip_paths, similarity_scores_clip)

    similar_images_deit_paths, similarity_scores_deit = prepare_csv_data(similar_images_deit[:5])
    print(similar_images_deit_paths, similarity_scores_deit)
    
    combined_clip_deit_paths, combined_clip_deit_scores = prepare_csv_data(combined_clip_deit)

    # Save results to separate CSV files
    save_to_csv("cnn_results.csv", QUERY_IMAGE_PATH, similar_images_cnn_paths, similarity_scores_cnn)
    save_to_csv("clip_results.csv", QUERY_IMAGE_PATH, similar_images_clip_paths, similarity_scores_clip)
    save_to_csv("deit_results.csv", QUERY_IMAGE_PATH, similar_images_deit_paths, similarity_scores_deit)
    save_to_csv("clip_deit_combined_results.csv", QUERY_IMAGE_PATH, combined_clip_deit_paths, combined_clip_deit_scores)




    # # Save results to separate CSV files
    # save_to_csv("cnn_results.csv", QUERY_IMAGE_PATH, similar_images_cnn_paths, similarity_scores_cnn)
    # save_to_csv("clip_results.csv", QUERY_IMAGE_PATH, similar_images_clip_paths, similarity_scores_clip)
    # save_to_csv("deit_results.csv", QUERY_IMAGE_PATH, similar_images_deit_paths, similarity_scores_deit)
    # save_to_csv("clip_deit_combined_results.csv", QUERY_IMAGE_PATH, combined_clip_deit_paths, combined_clip_deit_scores)

In [190]:
if __name__ == "__main__":
    main()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_6500\238729132.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=to

Query Image loaded

Extracting query image features using CNN model...
Features shape before reshape: (1000,)
Features shape after reshape: (1, 1000)

Extracting query image features using CLIP model...
Features shape before reshape: (512,)
Features shape after reshape: (1, 512)

Extracting query image features using DEIT model...
Features shape before reshape: (128,)
Features shape after reshape: (1, 128)

Leading pre-computed CNN Features File...
Features shape before reshape: (8158, 1, 1000)
Features shape after reshape: (8158, 1000)

Leading pre-computed CLIP Features File...
Features shape before reshape: (8158, 1, 768)
Reducing features from 768 to 512 using PCA
Features shape after reshape: (8158, 512)

Leading pre-computed DEIT Features File...
Features shape before reshape: (8158, 1, 128)
Features shape after reshape: (8158, 128)


AttributeError: 'numpy.ndarray' object has no attribute 'keys'